<a href="https://colab.research.google.com/github/Tomo-for-lab/Code-of-automating-DE/blob/main/create_original_description.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# general setting

# install & import library

In [ ]:
!pip install -q python-docx openai weave

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from google.colab import userdata
import os

from openai import OpenAI
from openai import AzureOpenAI
import json
import pandas as pd
import os
import glob
import numpy as np
from tqdm import tqdm
from google.colab import userdata


Change the working directory to the path of the current notebook

In [ ]:
from google.colab import drive
# Mount Google Drive to access files stored in it
drive.mount('/content/drive')


def get_notebook_path():
    # Get the path of the current Jupyter notebook
    kernel_id = re.search('kernel-(.*).json', ipykernel.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'), params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

def get_folder_path(folder_id):
    # Recursively get the full path of a folder given its ID
    if folder_id:
        folder = drive_service.files().get(fileId=folder_id, fields="name, parents").execute()
        folder_name = folder.get('name')
        parents = folder.get('parents')
        if parents:
            parent_path = get_folder_path(parents[0])
            return parent_path + '/' + folder_name
        else:
            return folder_name
    return ''

def get_file_path(file_id):
    # Recursively get the full path of a file given its ID
    file = drive_service.files().get(fileId=file_id, fields="name, parents").execute()
    file_name = file.get('name')
    parents = file.get('parents')
    if parents:
        parent_id = parents[0]
        parent_path = get_folder_path(parent_id)
        return parent_path
    else:
        return file_name

# Get the path of the current notebook
notebook_path = get_notebook_path()


# Authenticate and initialize the Google Drive API
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Extract the file ID from the notebook path (assumes file ID is part of the path)
file_id = re.search(r'fileId=(\w+)', notebook_path).group(1)

# Get the full path of the file using its ID
file_path = get_file_path(file_id)
if 'マイドライブ' in file_path:
    converted_path = re.sub(r'(^|/)マイドライブ($|/)', '/content/drive/MyDrive/', file_path)
elif 'MyDrive' in file_path:
    converted_path = re.sub(r'(^|/)MyDrive($|/)', '/content/drive/MyDrive/', file_path)
else:
    converted_path = '/content/drive/MyDrive/' + file_path

# Change the working directory to the converted path
os.chdir(converted_path)

Mounted at /content/drive


In [ ]:
import weave

weave.init("data_extraction_GPT_demo")

# Preparation of Azure for GPT

In [ ]:
#deploy names
deployment_ids = ["gpt-4o","gpt-4-turbo","gpt-3.5-turbo","text-embedding-ada-002"]

#If you use Azure
#key
API_KEY = userdata.get('Azure_API_key')
RESOURCE_ENDPOINT = userdata.get('RESOURCE_ENDPOINT')
AZURE_API_version = userdata.get('AZURE_API_version')

#client
client = AzureOpenAI(
    # https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#rest-api-versioning
    api_version=AZURE_API_version,
    # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal#create-a-resource
    azure_endpoint=RESOURCE_ENDPOINT,
    api_key=API_KEY
)



## Function for using the openAI API

In [ ]:
@weave.op() # 🐝
def GPT_chat(messages):
  response = client.chat.completions.create(
        model=deployment_ids[0],
        messages=messages,
        temperature=0
    )
  return response

#Data Extraction Manual

The following was prepared from the data extraction manual (docx).

The terminology was explained with reference to supplement.

Japanese converted to English.



In [ ]:
protocol = """
Data extraction tool for CBTICBNMA

Papers can be located by searching the “AuthorYYYY” folder in the IncludedTrials folder. The primary paper is named AuthorYYYY, the registry ULR AuthorYYYY_NCT1234, the protocol AuthorYYYY_pr, the appendix/supplement files AuthorYYY_app or AuthorYYY_supp, and the secondary papers AuthorYYYY_s1,2,…

When combining the results of two reviewers (say, AA and BB), make sure you name each data sheet differently (DE_AA_YYMMDD.xlsx) and you do NOT change those data.

#DE

<1. Characteristics of the studies>
Name of the study, year of publication, country, study site (single or multi-center), study design (individually-randomized or cluster-randomized), population characteristics (mean age, number of women, definition of insomnia, number of patients with primary insomnia), intervention (components, delivery format, qualification of therapists [if applicable], duration), outcomes (scale used for the primary outcome)

variable
description
Year: Publication year of the primary report. If only the results from registry are available, use the date when the results are posted instead.
Ind_clu: Choose “individual” if individually randomized, choose “cluster” if it is a cluster RCT.


ICC_for_cRCT: If it is a cluster RCT, extract Intra-cluster Correlation Coefficient. If it is not clearly stated, use “0.05”.

Country: Countries where the trial took place.

Single_multi: Whether the recruitment took place at a single or multi centers.

Age_mean: Mean age of the patients in the arm

Age_sd: SD of the above. SD=SE*sqrt(n), SD=(upper limit of 95%CI – mean)*sqrt(n)/1.96, SD= (Q3 − Q1)/1.35

Age_n: The number of patients used for calculating Age_mean.

F_n: Number of female in the arm

Hyp_n: Number of patients using hypnotics. If the drug treatment is part of the protocol treatment, “Hyp_n” should be equal to “n” (all the participants). If the drug treatment is not part of the protocol treatment but just allowed, we will use the number at baseline. We will prioritise prescribed drugs to OTC drugs.

Insomnia diagnosis: If more than one formal diagnostic criteria were mentioned, prioritise DSM, ICSD and then ICD. If you are using both ICD and DSM (if it says it matchs both), 1. choose formal_DSM (the one on top is preferred)
1. formal_DSM
1. formal_ICSD
1. formal_ICD
1. formal_Edinger2004
2. informal_ISI
2. informal_SCI
2. informal_PSQI
2. informal_AIS
2. informal_other_self_reported_scales
3. others

Primary insomnia_n: The number of primary insomnia patients (insomnia without comorbidities) in the arm. If it is focused on primary insomnia, it should be same as “n”.

Comorbidities:We will add phy_sleep_disorders to the complications category later, so please select phy_others and add OSA, etc. in the comments.
0. primary_insomnia
1. primary_secondary_mixed
2. psy_MDD
2. psy_anxiety
2. psy_PTSD
2. psy_schizophrenia_or_psychosis
2. psy_bipolar
2. psy_substance_dependence
2. psy_others
2. psy_mixed
3. phy_dialysis
3. phy_heart_failure
3. phy_diabetes
3. phy_fibromyalgia
3. phy_cancer
3. phy_steroid
3. phy_pregnancy_perinatal
3. phy_menopause
3. phy_others
3. phy_mixed
4. psy_phy_mixed



<2. Identification of components>
Choose “1” if the component is included, and choose “0” if it is not included.
TABLE 1 List of included components and their definitions.

Educational components (EDU)

se: Sleep hygiene education. General explanation about sleep (eg, sleep biology, characteristics of healthy sleep, changes in sleep patterns with aging, stress biology, and its impact on sleep) and general recommendations about lifestyle (eg, diet, exercise, substance use) and environmental factors (eg, light, noise, temperature) to improve sleep. This may include some elements of other components but those should not be the predominant part of the intervention.
General education about other symptoms or diseases were classified as non-specific treatment effect.

sd: Sleep diary. Self-monitoring of important daily sleep-related information using diary. This is
 considered to be present when clearly stated, or when sleep diary measures were reported. Wearable devices, polysomnography, actigraphy alone are not counted as sleep diary.

Cognitive components (COG)

cr: Cognitive restructuring. Skills to identify, challenge and change unhelpful beliefs about sleep that may disturb sleep. This may be simply called “cognitive therapy”. This may include behavioral experiments.

th: Third wave components. This includes mindfulness and acceptance and commitment therapy (ACT). Mindfulness is a form of meditation emphasizing a nonjudgmental state of heightened or complete awareness of one’s thoughts, emotions, or experiences on a moment-to-moment basis. ACT focuses on accepting the feelings and thoughts associated with insomnia through value-based behaviors. “Meditation” is included here.

cw: Constructive worry. Skills not to worry in bed by writing down the worries and their solutions before going to bed.

sr: Sleep restriction. Skills to improve sleep by limiting time in bed. First, time in bed is restricted to the average sleep duration and then it is increased or decreased depending on sleep efficiency.

sc: Stimulus control. Skills to re-associate the bed with sleep. Patients are instructed to; wake up at the same time every morning, refrain from daytime napping, go to bed only when sleepy, get out of bed when unable to sleep, and use the bed/bedroom for sleep and sex only.

re: Relaxation. Structured exercises designed to reduce somatic tension (eg, abdominal breathing, progressive muscle relaxation, autogenic training) and cognitive arousal (eg, guided imagery training).

pi: Paradoxical intention. Exercise to remain awake as long as possible after getting into bed.

w: Waiting component. Participants are aware that they can receive an active treatment after a waiting phase. If patients allocated to the waiting list control condition receive some other potentially therapeutic components, we will consider both the waiting component and the therapeutic components to be present.

dt: Conventional drug treatment. Rated positive when conventional drug treatment is present (drug treatment is part of the protocol treatment) or allowed (we will note the percentage of patients on drug). Recommended search term; “hypnotic,” “medication”

ns: Non-specific treatment effect. Effect of an intervention due to the patients’ belief that they are receiving some form of treatment. Miscellaneous skills not covered in other sections and not expected to have large effect (i.e. quasi-desensitization) are classified as having non-specific treatment effect. General education about other symptoms or diseases were classified as non-specific treatment effect.

Self-help, unguided, remote (NA): Default delivery format is self-help, unguided and remote, as it requires the least human resource.

he: Human encouragement. Reminders provided by human beings to proceed with the self-help, remote treatment program via telephone or email. This should not contain any support related to the therapeutic contents. Peer support such as discussion group will be regarded as this component. We will code this component separately from interaction with therapists (in, gp, ff) to see if adding human encouragement to self-help, remote interventions good enough to be effective.
A friendly reminder to "Good luck and please proceed with the program." Only consider with self-help ones.

tg: Therapeutic guidance. Therapeutic guidance in addition to self-help, remote interventions. This may be provided on a scheduled basis or as-needed basis. Technical support only is not included. We will code this component separately from interaction with therapists (in, gp, ff) to see if adding therapeutic guidance to self-help, remote interventions good enough to be effective.
Only consider those that are self-help. What is Guided Self-Help

ind: Individual. Individual interaction with therapists. The combination (in + ff) means individual, face-to-face sessions are held. The combination (in – ff) individual remote sessions, such as via telephone or videoconference.

gp: Group. Interaction with therapists as a member of a group.

ff: Face-to-face. Face-to-face interaction with therapists.

ae: Automatic encouragement. Automated reminders to proceed with the treatment program. This can be added both to the self-help interventions or the interventions including interactions with therapists. This should not contain any support related to the therapeutic contents.
Automatic reminders. Only consider with self-help ones.

others: If there is anything particular to state.

treatment_component,
EDU, COG, BEH,
treatment_group,
treatment_binary
These will be all automatically filled.


Duration_weeks: Duration of the intervention in weeks. If it is a self-help intervention that can be accessed anytime, we will regard the number of sessions as duration in weeks.


<3. Data to calculate effect sizes>
Agreement between the two reviewers with regard to the primary outcome (odds ratio) will be reported as
percentage agreement and intraclass correlation coefficient.

n: The number of patients randomized to the arm.

remission: The number of patients achieving remission at post-treatment or at its closest time point. If equidistant, we will use the longer timeframe.

def_remission_response: Copy & paste the definition of remission and response stated by the author. If multiple definitions are stated for different measures, copy & paste them all.

scale used: The scale used for the definition of remission. Prioritize; ISI > SCI > FOSQ > ESS > PSQI > AIS > other_self_reported_scales > WASO_SOL > WASO > SOL. We will use the definition stated by the author. Note that we will use sleep diary measures (subjective), not polysomnography measures (objective).

n_CSA: The number of patients analyzed for completer set analyses. Number of people who performed CBTI (Not Number of people for whom outcomes could be measured) by researcher definition.

r_CSA: The number of patients achieving remission in the completer set analyses.

Severity_bl_mean: Mean severity measured at baseline.

Severity_bl_sd: SD of the above. When only SE is given, please convert it to SD by calculating SE*n^(1/2).

Severity_bl_n: The number of patients measured their severity at baseline.

Severity_ch_mean: Extract this only when Severity_ep_mean, Severity _ep_sd and Sevrity_ep_n were not available.
Mean change of Severity from baseline to the endpoint (post-treatment or at its closest time point).
Change=Endpoint – Baseline. In case of improvement, ISI, PSQI and SOL, WASO will be negative values; SE and TST will be positive values.

Severity_ch_sd: Extract this only when Severity_ep_mean, Severity _ep_sd and Sevrity_ep_n were not available.
SD of the above. When only SE is given, please convert it to SD by calculating SE*n^(1/2).

Severity_ch_n: Extract this only when Severity_ep_mean, Severity _ep_sd and Sevrity_ep_n were not available.
The number of patients measured their severity change from baseline to the endpoint (post-treatment or at its closest time point).

Severity_ep_mean: Mean severity measured at post-treatment or at its closest time point.

Severity_ep_sd: SD of the above. When only SE is given, please convert it to SD by calculating SE*n^(1/2).

Severity_ep_n: The number of patients measured their severity at post-treatment or at its closest time point.
If ITT is used, this should be same as “n”.　Note, however, that in mITT, it may slightly differ from n. Note that "n" is the number of people randomly assigned, whereas the number of people extracted here is the number of people used to calculate mean and sd above.
"mixed model", "mixed model repeated measures (MMRM)", "linear mixed model", "growth curve analysis ", "hierarchical model" is used, and if the outcome being extracted is actually taken from LS means & SE calculated by MMRM, n is the number of people randomized, or the number of people included in the analysis.

Severity_scale: The scale used for measuring the sleep-related severity. Prioritize; ISI > SCI>  FOSQ > ESS > PSQI > AIS > other_self_reported_scales > WASO_SOL > WASO > SOL. We will use the definition stated by the author.
You may be using sleep quality rating as self_rating_other. This is closer to ISI than SOL or WASO, so use this. It is very important to know whether a larger or smaller number is better, so if you use something other than the number in the pull-down menu, please specify in the comments section whether you prefer a larger or smaller number.

r_weeks: Timepoint of the measurement of the primary outcome in weeks. If they measured at the end of the treatment, it will be ”0”. If they measured 4 weeks post-treatment, it will be “4”.

dropout: The number of patients dropped out from the study for any reason (=the number of patients whose outcome data not available). Those who are lost to follow-up (assessment at post-treatment) will be assumed to have dropped off.

SE_mean: Mean sleep efficiency in %.
Secondary outcomes are basically the values at the endpoint; for studies with change scores, enter the absolute value of baseline+change, and thus the SD is not known. For example, if the SE baseline is 72% and the change is +9% (SD 5%), enter 72+9=81 and use "*" for SD.

SE_sd:　Standard deviation of the above. When only SE is given, please convert it to SD by calculating SE*n^(1/2).

SE_n:　The number of patients measured for the numbers above. If ITT is used, this should be same as “n”.

TST_mean: Mean total sleep time in minutes.

TST_sd: Standard deviation of the above. When only SE is given, please convert it to SD by calculating SE*n^(1/2).

TST_n: The number of patients measured for the numbers above. If ITT is used, this should be same as “n”.

SOL_mean: Mean sleep onset latency in minutes.

SOL_sd: Standard deviation of the above. When only SE is given, please convert it to SD by calculating SE*n^(1/2).

SOL_n: The number of patients measured for the numbers above. If ITT is used, this should be same as “n”.

WASO_mean: Mean wake after sleep onset in minutes.

WASO_sd: When only SE is given, please convert it to SD by calculating SE*n^(1/2).

WASO_n: The number of patients measured for the numbers above. If ITT is used, this should be same as “n”.

Sleep_diary_weeks: Timepoint when the sleep diary was measured

r_long: The number of patients achieving remission at long-term follow-up (longest follow-up between 3 to 12 months)
The same point in time as the primary outcome may be adopted.

r_long_scale_used: Scales that were used for counting remission at long-term.

Severity_mean_long: Mean severity at long-term.

Severity_sd_long: Standard deviation of the above.

Severity_n_long: The number of patients measured for the numbers above. If ITT is used, this should be same as “n”.

Severity_ch_mean_long: Extract this only when Severity_mean_long, Severity _ sd_long and Sevrity_n_long were not available.

Severity_ch_sd_long: Extract this only when Severity_mean_long, Severity _ sd_long and Sevrity_n_long were not available.

Severity_ch_n_long: Extract this only when Severity_mean_long, Severity _ sd_long and Sevrity_n_long were not available.
If ITT is used, this should be same as “n”.

r_weeks_long: Timepoint of the measurement of the long-term follow-up in weeks.
Time of long-term F/U is weeks from baseline = duration of treatment + follow-up period (e.g., 6 weeks of treatment followed by 6 months of follow-up: 6+6*4=30 weeks)


SD=SE*sqrt(n)
　If mixed model repeated measures were used, n is the number of people randomized or included in the analysis.
SD=(upper limit of [100-x]%CI - mean)*sqrt(n)/ NORM.S.INV([100-x/2]/100)
  If 90%CI, then ((upper limit of 90%CI - mean)*sqrt(n)/ norm.s.inv(0.95))
If 95%CI, then ((upper limit of 95%CI - mean)*sqrt(n)/ norm.s.inv(0.975))
If 99%CI, then ((upper limit of 99%CI - mean)*sqrt(n)/ norm.s.inv(0.995))
SD= (Q3 − Q1)/1.35

"""

##Prepare variables of DE sheet

In [ ]:
# Load the Excel file
file_path = 'DE.xlsx'
df = pd.read_excel(file_path)

# Select the first row and drop any NaN values, then get the values as a list
selected_row = df.iloc[0].dropna().values

# Select the part of the row excluding RoB (Risk of Bias) related columns
_sheet_array = selected_row[0:92]

#In "study", "Arm", "Corresponding_Author", "Mail", "XXX", not enough data was extracted.
#"ir", "r_imputed", "r_imputed_long", "r_imputed_long_scale_used" are not present in the protocol
#"treatment_component", "EDU", "COG", "BEH", "treatment_group", "treatment_binary" are automatically calculated in the Excel file
#"Comment", "others" are excluded
excluded_variables = ["study", "Arm", "Corresponding_Author", "Mail", "ir", "XXX", "treatment_component", "EDU", "COG", "BEH", "treatment_group", "treatment_binary","r_imputed", "r_imputed_long", "r_imputed_long_scale_used", "Comment", "others"]

#'th', 'duration_weeks', 'n_CSA', 'r_CSA', 'r_weeks' were not included in the published version
excluded_variables.extend(['th', 'duration_weeks', 'n_CSA', 'r_CSA', 'r_weeks'])

# Create the final sheet array excluding the specified variables
sheet_array = [item for item in _sheet_array if item not in excluded_variables]

# Print the result
print(sheet_array)

['Year', 'ind_clu', 'ICC_for_cRCT', 'Country', 'Single_multi', 'Age_mean', 'Age_sd', 'Age_n', 'F_n', 'Hyp_n', 'Insomnia_diagnosis', 'Primary_insomnia_n', 'Comorbidities', 'se', 'sd', 'cr', 'cw', 'sr', 'sc', 're', 'pi', 'w', 'dt', 'ns', 'he', 'tg', 'ind', 'gp', 'ff', 'ae', 'n', 'remission', 'def_remission_response', 'scale_used', 'Severity_bl_mean', 'Severity_bl_sd', 'Severity_bl_n', 'Severity_ch_mean', 'Severity_ch_sd', 'Severity_ch_n', 'Severity_ep_mean', 'Severity_ep_sd', 'Severity_ep_n', 'Severity_scale', 'dropout', 'SE_mean', 'SE_sd', 'SE_n', 'TST_mean', 'TST_sd', 'TST_n', 'SOL_mean', 'SOL_sd', 'SOL_n', 'WASO_mean', 'WASO_sd', 'WASO_n', 'Sleep_diary_weeks', 'r_long', 'r_long_scale_used', 'Severity_mean_long', 'Severity_sd_long', 'Severity_n_long', 'Severity_ch_mean_long', 'Severity_ch_sd_long', 'Severity_ch_n_long', 'r_weeks_long']


In [ ]:
print(sheet_array[29])

ae


##To create original descriptions


In [ ]:
# Define the system message content
system_content = """
You are a systematic reviewer. I want you to write a JSON structure to extract the variables of a full text article for function calling. You should delete the notes or memos columns.
Do not change the number of columns and those names.
You have to write descriptions in as much details as you can according to the inputted data extraction protocol.
You can include concrete examples used in the protocol.
Include the following three perspectives: what kind of data you extracted, when the extracted data was generated (randomized, baseline, post-treatment, etc.), and the arm from which it was extracted.

You can write the contents of the protocol as is, or you can supplement the text to make it easier for you to understand when extracting the data.
The type of data extraction is number or string.
"""

# Define the user message content
user_content = """
You are a systematic reviewer. I want you to write a JSON structure to extract the variables of a full text article for function calling. You should delete the notes or memos columns.
Do not change the number of columns and those names.
You have to write descriptions in as much details as you can according to the inputted data extraction protocol.
Include the following three perspectives: what kind of data you extracted, when the extracted data was generated (randomized, baseline, post-treatment, etc.), and the arm from which it was extracted.

You can write the contents of the protocol as is, or you can supplement the text to make it easier for you to understand when extracting the data.
The type of data extraction is number or string.

example:

        {
          "name": "name of the variable",
          "type": "string",
          "description": "the explation of what the variable is and how you extract the variable. "
        }

The columns names are as follows:
"""

# Add the first 30 column names to the user content
for i in range(30):
  user_content += str(sheet_array[i]) + ", "

# Add a reference to the data extraction protocol
user_content += """
this is the data extraction protocol. Please use to write the JSON format.
"""

user_content += protocol

In [ ]:
weave.init("creating_original_description")

In [ ]:
# Create the message list for the GPT function call
messages=[]
messages.append({"role": "system", "content": system_content})
messages.append({"role": "user", "content": user_content})

# Call the GPT function to generate the response
response_1 = GPT_chat(messages)

In [ ]:
response_1

ChatCompletion(id='chatcmpl-9YP45LANjbW7TXCeCZsayV5sxy9Hq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n  {\n    "name": "Year",\n    "type": "number",\n    "description": "Publication year of the primary report. If only the results from registry are available, use the date when the results are posted instead."\n  },\n  {\n    "name": "ind_clu",\n    "type": "string",\n    "description": "Study design: Choose \'individual\' if individually randomized, choose \'cluster\' if it is a cluster RCT."\n  },\n  {\n    "name": "ICC_for_cRCT",\n    "type": "number",\n    "description": "If it is a cluster RCT, extract Intra-cluster Correlation Coefficient. If it is not clearly stated, use \'0.05\'."\n  },\n  {\n    "name": "Country",\n    "type": "string",\n    "description": "Countries where the trial took place."\n  },\n  {\n    "name": "Single_multi",\n    "type": "string",\n    "description": "Whether the recruitment took p

In [ ]:
system_content = """
You are a systematic reviewer. I want you to write a JSON structure to extract the variables of a full text article for function calling. You should delete the notes or memos columns.
Do not change the number of columns and those names.
You have to write descriptions in as much details as you can according to the inputted data extraction protocol.

You can write the contents of the protocol as is, or you can supplement the text to make it easier for you to understand when extracting the data.
The type of data extraction is number or string.
"""
user_content = """
You are a systematic reviewer. I want you to write a JSON structure to extract the variables of a full text article for function calling. You should delete the notes or memos columns.
Do not change the number of columns and those names.
You have to write descriptions in as much details as you can according to the inputted data extraction protocol.

You can write the contents of the protocol as is, or you can supplement the text to make it easier for you to understand when extracting the data.
The type of data extraction is number or string.

example:

        {
          "name": "name of the variable",
          "type": "string",
          "description": "the explation of what the variable is and how you extract the variable"
        }

The columns names are as follows:
"""

# Add the remaining 30 column names to the user content
for i in range(30, len(sheet_array)):
  user_content += str(sheet_array[i]) + ", "

user_content += """
this is the data extraction protocol. Please use to write the JSON format.
"""

user_content += protocol

In [ ]:
weave.init("creating_original_description")

In [ ]:
messages=[]
messages.append({"role": "system", "content": system_content})
messages.append({"role": "user", "content": user_content})

response_2 = GPT_chat(messages)

In [ ]:
response_2

ChatCompletion(id='chatcmpl-9TBCZxnWSDJSSt15PgW30cFJYpgXQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```json\n[\n  {\n    "name": "n",\n    "type": "number",\n    "description": "The number of patients randomized to the arm."\n  },\n  {\n    "name": "remission",\n    "type": "number",\n    "description": "The number of patients achieving remission at post-treatment or at its closest time point. If equidistant, use the longer timeframe."\n  },\n  {\n    "name": "def_remission_response",\n    "type": "string",\n    "description": "Copy & paste the definition of remission and response stated by the author. If multiple definitions are stated for different measures, copy & paste them all."\n  },\n  {\n    "name": "scale_used",\n    "type": "string",\n    "description": "The scale used for the definition of remission. Prioritize; ISI > SCI > FOSQ > ESS > PSQI > AIS > other_self_reported_scales > WASO_SOL > WASO > SOL. Use the defini

In [ ]:
#Integrate the obtained responses
output ="""
[\n  {\n    "name": "Year",\n    "type": "number",\n    "description": "Publication year of the primary report. If only the results from registry are available, use the date when the results are posted instead."\n  },\n  {\n    "name": "ind_clu",\n    "type": "string",\n    "description": "Study design: Choose \'individual\' if individually randomized, choose \'cluster\' if it is a cluster RCT."\n  },\n  {\n    "name": "ICC_for_cRCT",\n    "type": "number",\n    "description": "Intra-cluster Correlation Coefficient for cluster RCTs. If it is not clearly stated, use \'0.05\'."\n  },\n  {\n    "name": "Country",\n    "type": "string",\n    "description": "Countries where the trial took place."\n  },\n  {\n    "name": "Single_multi",\n    "type": "string",\n    "description": "Whether the recruitment took place at a single or multi centers."\n  },\n  {\n    "name": "Age_mean",\n    "type": "number",\n    "description": "Mean age of the patients in the arm."\n  },\n  {\n    "name": "Age_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean age. SD can be calculated using SE or CI if not directly provided."\n  },\n  {\n    "name": "Age_n",\n    "type": "number",\n    "description": "The number of patients used for calculating Age_mean."\n  },\n  {\n    "name": "F_n",\n    "type": "number",\n    "description": "Number of female patients in the arm."\n  },\n  {\n    "name": "Hyp_n",\n    "type": "number",\n    "description": "Number of patients using hypnotics. If the drug treatment is part of the protocol treatment, \'Hyp_n\' should be equal to \'n\'. If not, use the number at baseline."\n  },\n  {\n    "name": "Insomnia_diagnosis",\n    "type": "string",\n    "description": "Formal or informal diagnostic criteria used for insomnia. Prioritize DSM, ICSD, ICD, and other formal criteria over informal scales."\n  },\n  {\n    "name": "Primary_insomnia_n",\n    "type": "number",\n    "description": "The number of primary insomnia patients (insomnia without comorbidities) in the arm. If focused on primary insomnia, it should be same as \'n\'."\n  },\n  {\n    "name": "Comorbidities",\n    "type": "string",\n    "description": "Comorbid conditions present in the study population. Use predefined categories such as primary_insomnia, psy_MDD, phy_heart_failure, etc."\n  },\n  {\n    "name": "se",\n    "type": "number",\n    "description": "Sleep hygiene education component: 1 if included, 0 if not."\n  },\n  {\n    "name": "sd",\n    "type": "number",\n    "description": "Sleep diary component: 1 if included, 0 if not."\n  },\n  {\n    "name": "cr",\n    "type": "number",\n    "description": "Cognitive restructuring component: 1 if included, 0 if not."\n  },\n  {\n    "name": "cw",\n    "type": "number",\n    "description": "Constructive worry component: 1 if included, 0 if not."\n  },\n  {\n    "name": "sr",\n    "type": "number",\n    "description": "Sleep restriction component: 1 if included, 0 if not."\n  },\n  {\n    "name": "sc",\n    "type": "number",\n    "description": "Stimulus control component: 1 if included, 0 if not."\n  },\n  {\n    "name": "re",\n    "type": "number",\n    "description": "Relaxation component: 1 if included, 0 if not."\n  },\n  {\n    "name": "pi",\n    "type": "number",\n    "description": "Paradoxical intention component: 1 if included, 0 if not."\n  },\n  {\n    "name": "w",\n    "type": "number",\n    "description": "Waiting component: 1 if included, 0 if not."\n  },\n  {\n    "name": "dt",\n    "type": "number",\n    "description": "Conventional drug treatment component: 1 if included, 0 if not."\n  },\n  {\n    "name": "ns",\n    "type": "number",\n    "description": "Non-specific treatment effect component: 1 if included, 0 if not."\n  },\n  {\n    "name": "he",\n    "type": "number",\n    "description": "Human encouragement component: 1 if included, 0 if not."\n  },\n  {\n    "name": "tg",\n    "type": "number",\n    "description": "Therapeutic guidance component: 1 if included, 0 if not."\n  },\n  {\n    "name": "ind",\n    "type": "number",\n    "description": "Individual interaction with therapists: 1 if included, 0 if not."\n  },\n  {\n    "name": "gp",\n    "type": "number",\n    "description": "Group interaction with therapists: 1 if included, 0 if not."\n  },\n  {\n    "name": "ff",\n    "type": "number",\n    "description": "Face-to-face interaction with therapists: 1 if included, 0 if not."\n  },\n  {\n    "name": "ae",\n    "type": "number",\n    "description": "Automatic encouragement component: 1 if included, 0 if not."\n  },\n
{\n    "name": "n",\n    "type": "number",\n    "description": "The number of patients randomized to the arm."\n  },\n  {\n    "name": "remission",\n    "type": "number",\n    "description": "The number of patients achieving remission at post-treatment or at its closest time point. If equidistant, use the longer timeframe."\n  },\n  {\n    "name": "def_remission_response",\n    "type": "string",\n    "description": "Copy & paste the definition of remission and response stated by the author. If multiple definitions are stated for different measures, copy & paste them all."\n  },\n  {\n    "name": "scale_used",\n    "type": "string",\n    "description": "The scale used for the definition of remission. Prioritize; ISI > SCI > FOSQ > ESS > PSQI > AIS > other_self_reported_scales > WASO_SOL > WASO > SOL. Use the definition stated by the author."\n  },\n  {\n    "name": "Severity_bl_mean",\n    "type": "number",\n    "description": "Mean severity measured at baseline."\n  },\n  {\n    "name": "Severity_bl_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean severity measured at baseline. When only SE is given, convert it to SD by calculating SE*n^(1/2)."\n  },\n  {\n    "name": "Severity_bl_n",\n    "type": "number",\n    "description": "The number of patients measured for their severity at baseline."\n  },\n  {\n    "name": "Severity_ch_mean",\n    "type": "number",\n    "description": "Mean change of severity from baseline to the endpoint (post-treatment or at its closest time point). Extract this only when Severity_ep_mean, Severity_ep_sd, and Severity_ep_n are not available."\n  },\n  {\n    "name": "Severity_ch_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean change of severity from baseline to the endpoint. Extract this only when Severity_ep_mean, Severity_ep_sd, and Severity_ep_n are not available. When only SE is given, convert it to SD by calculating SE*n^(1/2)."\n  },\n  {\n    "name": "Severity_ch_n",\n    "type": "number",\n    "description": "The number of patients measured for their severity change from baseline to the endpoint. Extract this only when Severity_ep_mean, Severity_ep_sd, and Severity_ep_n are not available."\n  },\n  {\n    "name": "Severity_ep_mean",\n    "type": "number",\n    "description": "Mean severity measured at post-treatment or at its closest time point."\n  },\n  {\n    "name": "Severity_ep_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean severity measured at post-treatment or at its closest time point. When only SE is given, convert it to SD by calculating SE*n^(1/2)."\n  },\n  {\n    "name": "Severity_ep_n",\n    "type": "number",\n    "description": "The number of patients measured for their severity at post-treatment or at its closest time point."\n  },\n  {\n    "name": "Severity_scale",\n    "type": "string",\n    "description": "The scale used for measuring the sleep-related severity. Prioritize; ISI > SCI > FOSQ > ESS > PSQI > AIS > other_self_reported_scales > WASO_SOL > WASO > SOL. Use the definition stated by the author."\n  },\n  {\n    "name": "dropout",\n    "type": "number",\n    "description": "The number of patients who dropped out from the study for any reason. Those who are lost to follow-up will be assumed to have dropped off."\n  },\n  {\n    "name": "SE_mean",\n    "type": "number",\n    "description": "Mean sleep efficiency in percentage."\n  },\n  {\n    "name": "SE_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean sleep efficiency. When only SE is given, convert it to SD by calculating SE*n^(1/2)."\n  },\n  {\n    "name": "SE_n",\n    "type": "number",\n    "description": "The number of patients measured for sleep efficiency. If ITT is used, this should be the same as \'n\'."\n  },\n  {\n    "name": "TST_mean",\n    "type": "number",\n    "description": "Mean total sleep time in minutes."\n  },\n  {\n    "name": "TST_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean total sleep time. When only SE is given, convert it to SD by calculating SE*n^(1/2)."\n  },\n  {\n    "name": "TST_n",\n    "type": "number",\n    "description": "The number of patients measured for total sleep time. If ITT is used, this should be the same as \'n\'."\n  },\n  {\n    "name": "SOL_mean",\n    "type": "number",\n    "description": "Mean sleep onset latency in minutes."\n  },\n  {\n    "name": "SOL_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean sleep onset latency. When only SE is given, convert it to SD by calculating SE*n^(1/2)."\n  },\n  {\n    "name": "SOL_n",\n    "type": "number",\n    "description": "The number of patients measured for sleep onset latency. If ITT is used, this should be the same as \'n\'."\n  },\n  {\n    "name": "WASO_mean",\n    "type": "number",\n    "description": "Mean wake after sleep onset in minutes."\n  },\n  {\n    "name": "WASO_sd",\n    "type": "number",\n    "description": "Standard deviation of the mean wake after sleep onset. When only SE is given, convert it to SD by calculating SE*n^(1/2)."\n  },\n  {\n    "name": "WASO_n",\n    "type": "number",\n    "description": "The number of patients measured for wake after sleep onset. If ITT is used, this should be the same as \'n\'."\n  },\n  {\n    "name": "Sleep_diary_weeks",\n    "type": "number",\n    "description": "Timepoint when the sleep diary was measured in weeks."\n  },\n  {\n    "name": "r_long",\n    "type": "number",\n    "description": "The number of patients achieving remission at long-term follow-up (longest follow-up between 3 to 12 months). The same point in time as the primary outcome may be adopted."\n  },\n  {\n    "name": "r_long_scale_used",\n    "type": "string",\n    "description": "Scales that were used for counting remission at long-term."\n  },\n  {\n    "name": "Severity_mean_long",\n    "type": "number",\n    "description": "Mean severity at long-term follow-up."\n  },\n  {\n    "name": "Severity_sd_long",\n    "type": "number",\n    "description": "Standard deviation of the mean severity at long-term follow-up."\n  },\n  {\n    "name": "Severity_n_long",\n    "type": "number",\n    "description": "The number of patients measured for severity at long-term follow-up. If ITT is used, this should be the same as \'n\'."\n  },\n  {\n    "name": "Severity_ch_mean_long",\n    "type": "number",\n    "description": "Mean change of severity from baseline to long-term follow-up. Extract this only when Severity_mean_long, Severity_sd_long, and Severity_n_long are not available."\n  },\n  {\n    "name": "Severity_ch_sd_long",\n    "type": "number",\n    "description": "Standard deviation of the mean change of severity from baseline to long-term follow-up. Extract this only when Severity_mean_long, Severity_sd_long, and Severity_n_long are not available."\n  },\n  {\n    "name": "Severity_ch_n_long",\n    "type": "number",\n    "description": "The number of patients measured for severity change from baseline to long-term follow-up. Extract this only when Severity_mean_long, Severity_sd_long, and Severity_n_long are not available. If ITT is used, this should be the same as \'n\'."\n  },\n  {\n    "name": "r_weeks_long",\n    "type": "number",\n    "description": "Timepoint of the measurement of the long-term follow-up in weeks. Time of long-term follow-up is weeks from baseline = duration of treatment + follow-up period (e.g., 6 weeks of treatment followed by 6 months of follow-up: 6+6*4=30 weeks)."\n  }\n]\n
"""

In [ ]:
# Parse the JSON output from the GPT response
output_json = json.loads(output)
output_json

[{'name': 'Year',
  'type': 'number',
  'description': 'Publication year of the primary report. If only the results from registry are available, use the date when the results are posted instead.'},
 {'name': 'ind_clu',
  'type': 'string',
  'description': "Study design: Choose 'individual' if individually randomized, choose 'cluster' if it is a cluster RCT."},
 {'name': 'ICC_for_cRCT',
  'type': 'number',
  'description': "Intra-cluster Correlation Coefficient for cluster RCTs. If it is not clearly stated, use '0.05'."},
 {'name': 'Country',
  'type': 'string',
  'description': 'Countries where the trial took place.'},
 {'name': 'Single_multi',
  'type': 'string',
  'description': 'Whether the recruitment took place at a single or multi centers.'},
 {'name': 'Age_mean',
  'type': 'number',
  'description': 'Mean age of the patients in the arm.'},
 {'name': 'Age_sd',
  'type': 'number',
  'description': 'Standard deviation of the mean age. SD can be calculated using SE or CI if not dire

Give a choice in the description for variables for which a pull-down choice is given in the DE sheet.

In [ ]:
for item in output_json:
  if item['name'] == 'ind_clu':
    item['description'] += ' You have to choose from the options [individual, cluster].'
  if item['name'] == 'Single_multi':
    item['description'] += ' You have to choose from the options [single, multi].'
  if item['name'] == 'Insomnia_diagnosis':
    item['description'] += ' You have to choose from the options [1. formal_DSM, 1. formal_ICSD, 1. formal_ICD, 1. formal_Edinger2004, 2. informal_ISI, 2. informal_SCI, 2. informal_PSQI, 2. informal_AIS, 2. informal_other_self_reported_scales, 3. others].'
  if item['name'] == 'Comorbidities':
    item['description'] += ' You have to choose from the options [0. primary_insomnia, 1. primary_secondary_mixed, 2. psy_MDD, 2. psy_anxiety, 2. psy_PTSD, 2. psy_schizophrenia_or_psychosis, 2. psy_bipolar, 2. psy_substance_dependence, 2. psy_others, 2. psy_mixed, 3. phy_dialysis, 3. phy_heart_failure, 3. phy_diabetes, 3. phy_fibromyalgia, 3. phy_cancer, 3. phy_steroid, 3. phy_pregnancy_perinatal, 3. phy_menopause, 3. phy_others, 3. phy_mixed, 4. psy_phy_mixed]'
  if item['name'] == 'scale_used':
    item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]'
  if item['name'] == 'Severity_scale':
    item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]'
  if item['name'] == 'r_long_scale_used':
    item['description'] += ' You have to choose from the options [ISI, SCI, FOSQ, ESS, PSQI, AIS, other_self_reported_scales, WASO_SOL, WASO, SOL]'

output_json

[{'name': 'Year',
  'type': 'number',
  'description': 'Publication year of the primary report. If only the results from registry are available, use the date when the results are posted instead.'},
 {'name': 'ind_clu',
  'type': 'string',
  'description': "Study design: Choose 'individual' if individually randomized, choose 'cluster' if it is a cluster RCT. You have to choose from the options [individual, cluster]."},
 {'name': 'ICC_for_cRCT',
  'type': 'number',
  'description': "Intra-cluster Correlation Coefficient for cluster RCTs. If it is not clearly stated, use '0.05'."},
 {'name': 'Country',
  'type': 'string',
  'description': 'Countries where the trial took place.'},
 {'name': 'Single_multi',
  'type': 'string',
  'description': 'Whether the recruitment took place at a single or multi centers. You have to choose from the options [single, multi].'},
 {'name': 'Age_mean',
  'type': 'number',
  'description': 'Mean age of the patients in the arm.'},
 {'name': 'Age_sd',
  'type':

Verify that the output "name" is the same as the original variable name.

In [ ]:
# Extract the 'name' values from the JSON data
json_names = {item['name'] for item in output_json}

# Compare the names from the list and the JSON data
in_json_not_in_list = json_names - set(sheet_array)
in_list_not_in_json = set(sheet_array) - json_names

# Display the results
print("JSONにのみ存在するnameの値:", in_json_not_in_list) # Names present only in the JSON data
print("リストにのみ存在する要素:", in_list_not_in_json) # Elements present only in the list

JSONにのみ存在するnameの値: set()
リストにのみ存在する要素: set()


Change the type of variable to number.

In [ ]:
for item in output_json:
    if item['type'] == 'integer' or item['type'] == 'float':
        item['type'] = 'number'

In [ ]:
unique_types = {item['type'] for item in output_json}
print(unique_types)

{'string', 'number'}


In [ ]:
# save the result as json file
output_path = 'original_desciption_insomnia.json'
try:
  with open(output_path, 'w') as f:
      json.dump(output_json, f, indent=4)
except:
  print("ERROR: output string is not in json style!")